## Verifying Datasets 2/16/19

In [2]:
import os
import pandas as pd
import numpy as np
from data_functions import *
from sklearn.metrics import log_loss, accuracy_score

df_1 = pd.read_csv('match_data_constructed/match_df_6_28_2018.csv')
df_2 = pd.read_csv('match_data_constructed/match_df_02_16_2019.csv')
# os.listdir('match_data_constructed/')

In [30]:
df_2[[col for col in df_2.columns if col not in df_1.columns]]

,sf_avg_52_s,sf_avg_52_r,w_52_commop_s_pct,w_52_commop_r_pct,l_52_commop_s_pct,l_52_commop_r_pct,p0_sf_52_swon,p1_sf_52_swon,p0_sf_52_svpt,p1_sf_52_svpt,...,p0_sf_52_rpt,p1_sf_52_rpt,p0_sf_s_pct,p0_sf_r_pct,p1_sf_s_pct,p1_sf_r_pct,p0_s_sf_kls,p1_s_sf_kls,match_prob_sf_kls,sf_elo_prob
0,0.616020,0.383980,0.653095,0.453095,0.546905,0.346905,681.0,956.0,1077.0,1453.0,...,1107.0,1486.0,0.632312,0.376694,0.657949,0.459623,0.574403,0.682969,8.607655e-02,0.030039
1,0.616020,0.383980,0.599580,0.399580,0.600420,0.400420,567.0,1034.0,918.0,1678.0,...,943.0,1678.0,0.617647,0.400848,0.616210,0.424315,0.595046,0.617075,3.883966e-01,0.408993
2,0.616020,0.383980,0.659367,0.459367,0.540633,0.340633,956.0,0.0,1453.0,0.0,...,1486.0,0.0,0.657949,0.459623,0.645765,0.354235,0.705428,0.587856,9.267578e-01,0.993750
3,0.616020,0.383980,0.600000,0.400000,0.600000,0.400000,1034.0,0.0,1678.0,0.0,...,1678.0,0.0,0.616210,0.424315,0.645765,0.354235,0.663689,0.623164,6.927686e-01,0.935964
4,0.638217,0.361783,0.600000,0.400000,0.600000,0.400000,1713.0,212.0,2382.0,352.0,...,2509.0,349.0,0.719144,0.357513,0.602273,0.335244,0.735352,0.596212,9.537231e-01,0.959601
5,0.638291,0.361709,0.637785,0.437785,0.562215,0.362215,315.0,930.0,465.0,1535.0,...,450.0,1468.0,0.677419,0.331111,0.605863,0.365804,0.662920,0.626057,6.764321e-01,0.593244
6,0.638288,0.361712,0.666700,0.466700,0.533300,0.333300,690.0,1010.0,1150.0,1590.0,...,1166.0,1596.0,0.600000,0.380789,0.635220,0.357769,0.593541,0.605741,4.372968e-01,0.306469
7,0.638266,0.361734,0.600000,0.400000,0.600000,0.400000,1273.0,0.0,1916.0,0.0,...,1940.0,0.0,0.664405,0.350000,0.645765,0.354235,0.661524,0.647119,5.702797e-01,0.887387
8,0.638304,0.361696,0.600000,0.400000,0.600000,0.400000,0.0,1507.0,0.0,2311.0,...,0.0,2470.0,0.645765,0.354235,0.652099,0.368826,0.628217,0.649142,3.968814e-01,0.114080
9,0.638328,0.361672,0.578242,0.378242,0.621758,0.421758,840.0,966.0,1241.0,1499.0,...,1233.0,1505.0,0.676873,0.368208,0.644430,0.387375,0.640728,0.627452,5.661714e-01,0.509748


In [21]:
cols = [u'match_year',
       u'match_month', u'avg_52_s', u'avg_52_r', u'tny_stats', u'winner',
       u'p0_elo_538', u'p1_elo_538', u'p0_sf_elo_538',
       u'p1_sf_elo_538', u'p0_swon', u'p1_swon', u'p0_svpt', u'p1_svpt',
       u'p0_rwon', u'p1_rwon', u'p0_rpt', u'p1_rpt', u'p0_52_swon',
       u'p1_52_swon', u'p0_52_svpt', u'p1_52_svpt', u'p0_52_rwon',
       u'p1_52_rwon', u'p0_52_rpt', u'p1_52_rpt', u'p0_52_s_adj',
       u'p1_52_s_adj', u'p0_52_r_adj', u'p1_52_r_adj', u'p0_s_pct',
       u'p0_r_pct', u'p1_s_pct', u'p1_r_pct', u'elo_diff', u'sf_elo_diff',
       u'p0_s_pct_obsv', u'p1_s_pct_obsv', u'p0_s_pct_EM', u'p1_s_pct_EM',
       u'p0_r_pct_EM', u'p1_r_pct_EM', u'p0_52_s_adj_EM', u'p1_52_s_adj_EM',
       u'p0_52_r_adj_EM', u'p1_52_r_adj_EM', u'match_id', u'p0_s_kls',
       u'p1_s_kls', u'p0_s_kls_EM', u'p1_s_kls_EM', u'p0_s_adj_kls',
       u'p1_s_adj_kls', u'p0_s_adj_kls_EM', u'p1_s_adj_kls_EM',
       u'match_prob_kls', u'match_prob_kls_EM', u'match_prob_adj_kls',
       u'match_prob_adj_kls_EM', u'elo_prob', u's_total', u'p0_s_kls_elo',
       u'p1_s_kls_elo']

In [22]:
i = 23000
for col in cols:
    print col, np.sum(np.isclose(df_1[col], df_2[col]))

match_year 23270
match_month 23270
avg_52_s 23270
avg_52_r 23270
tny_stats 23270
winner 11563
p0_elo_538 11578
p1_elo_538 11578
p0_sf_elo_538 11597
p1_sf_elo_538 11597
p0_swon 10993
p1_swon 10993
p0_svpt 10900
p1_svpt 10900
p0_rwon 10872
p1_rwon 10872
p0_rpt 10900
p1_rpt 10900
p0_52_swon 11935
p1_52_swon 11935
p0_52_svpt 11933
p1_52_svpt 11933
p0_52_rwon 11939
p1_52_rwon 11939
p0_52_rpt 11934
p1_52_rpt 11934
p0_52_s_adj 11932
p1_52_s_adj 11932
p0_52_r_adj 11932
p1_52_r_adj 11932
p0_s_pct 11932
p0_r_pct 11933
p1_s_pct 11932
p1_r_pct 11933
elo_diff 11576
sf_elo_diff 11597
p0_s_pct_obsv 12507
p1_s_pct_obsv 12507
p0_s_pct_EM 11934
p1_s_pct_EM 11934
p0_r_pct_EM 11932
p1_r_pct_EM 11932
p0_52_s_adj_EM 11932
p1_52_s_adj_EM 11932
p0_52_r_adj_EM 11932
p1_52_r_adj_EM 11932
match_id 23270
p0_s_kls 11933
p1_s_kls 11933
p0_s_kls_EM 11935
p1_s_kls_EM 11935
p0_s_adj_kls 11933
p1_s_adj_kls 11933
p0_s_adj_kls_EM

KeyError: u'p0_s_adj_kls_EM'

In [11]:
df_1.loc[23000]

tny_id                             2018-580
tny_name                    Australian Open
surface                                Hard
draw_size                               128
tny_date                         2018-01-15
match_num                               146
score                       6-2 6-7 7-6 6-4
best_of                                   5
is_gs                                  True
match_year                             2018
match_month                               1
avg_52_s                           0.635211
avg_52_r                           0.364789
tny_stats                          0.637466
winner                                    0
p0_name                     Daniil Medvedev
p1_name                  Thanasi Kokkinakis
p0_elo_538                          1949.17
p1_elo_538                          1798.69
p0_sf_elo_538                       1839.61
p1_sf_elo_538                       1702.06
p0_swon                                 100
p1_swon                         

In [6]:
df_2.loc[23000]

tny_id                          2018-580
tny_name                 Australian Open
surface                             Hard
draw_size                            128
tny_date                      2018-01-15
match_num                            146
score                    6-2 6-7 7-6 6-4
best_of                                5
is_gs                               True
match_year                          2018
match_month                            1
avg_52_s                        0.635211
avg_52_r                        0.364789
sf_avg_52_s                     0.636352
sf_avg_52_r                     0.363648
tny_stats                       0.637466
w_52_commop_s_pct               0.604087
w_52_commop_r_pct               0.404087
l_52_commop_s_pct               0.595913
l_52_commop_r_pct               0.395913
winner                                 0
p0_name                  Daniil Medvedev
p1_name               Thanasi Kokkinakis
p0_elo_538                       1949.17
p1_elo_538      

In [3]:
df_1.shape, df_2.shape

((23270, 74), (23270, 93))

In [35]:
# a test_df for 
test_df = df[(df.match_year >= 2012) & (df.match_year <= 2015)].dropna().reset_index()
test_df = test_df[test_cols]

In [37]:
test_df.head()

,tny_id,tny_name,surface,draw_size,tny_date,match_num,score,best_of,is_gs,match_year,match_month,winner,p0_swon,p1_swon,p0_svpt,p1_svpt,p0_rwon,p1_rwon,p0_rpt,p1_rpt
0,2012-339,Brisbane,Hard,32,2012-01-01,1,5-7 6-3 6-2,3,False,2012,1,1,44.0,58.0,77.0,92.0,34.0,33.0,92.0,77.0
1,2012-339,Brisbane,Hard,32,2012-01-01,2,6-7 7-6 6-3,3,False,2012,1,0,78.0,74.0,104.0,103.0,29.0,26.0,103.0,104.0
2,2012-339,Brisbane,Hard,32,2012-01-01,3,7-6 6-4,3,False,2012,1,1,53.0,52.0,79.0,71.0,19.0,26.0,71.0,79.0
3,2012-339,Brisbane,Hard,32,2012-01-01,4,3-6 6-1 6-4,3,False,2012,1,0,47.0,50.0,75.0,90.0,40.0,28.0,90.0,75.0
4,2012-339,Brisbane,Hard,32,2012-01-01,6,6-3 7-5,3,False,2012,1,1,43.0,46.0,77.0,66.0,20.0,34.0,66.0,77.0


In [38]:
generate_52_stats(test_df, 0)

KeyError: 'w_name'

In [8]:
# TODO: verify datasets

In [ ]:
## NOTE: you made a mistake implementing EM estimates...double check this 
## looks like generate_EM() is not being run correctly
## might actually be due to only calculating estimates for relevant years and using entire match 
## frame; this might affect the strength

## TO DO: split helper functions into generating elo, then getting 52_stats
## need to make sure that df has all relevant matches (within year range) prior to generating
## EM stats, otherwise the value of 'n' will be thrown off

In [3]:
import pandas as pd
import numpy as np
from data_functions import *
from sklearn.metrics import log_loss, accuracy_score

df = pd.read_csv('match_data_constructed/match_df_6_29_2018')
f_path = '../../repositories/college/research/my_data/elo_atp_matches_all_10_29.csv'
df_old = pd.read_csv(f_path)

In [4]:
1513/3964.

0.3816851664984864

In [5]:
f_t, f_av = .6153, .6468
g_av = 1-f_av
f_i, g_j = .6465, .3478

f_t + (f_i-f_av) - (g_j-g_av)

0.6203999999999998

In [6]:
match_sub = df[(df['p0_name']=='Kevin Anderson') & (df['p1_name']=='Fernando Verdasco')]
match_sub = match_sub[match_sub['match_year']==2013]
# match_sub = match_sub[['p0_name','p1_name','p0_52_rwon','p0_52_rpt','p1_52_rwon','p1_52_rpt']]


In [10]:
df = df[df['tny_name']!='ATP Challenger Tour Finals CH']
df = df[df['tny_name']!='Davis Cup']
df_old = df_old[df_old['tny_name']!='ATP Challenger Tour Finals CH']
df_old = df_old[df_old['tny_name']!='Davis Cup']

YEAR = 2014
sub = df[df['match_year']==YEAR].reset_index()
sub_old = df_old[df_old['match_year']==YEAR].reset_index()

In [14]:
sub.shape

(2488, 75)

In [12]:
a = 0
b = 0
for tny_name in set(sub['tny_name']):
    a += sum(sub['tny_name']==tny_name)
    b += sum(sub_old['tny_name']==tny_name)
    if sum(sub_old['tny_name']==tny_name) != sum(sub['tny_name']==tny_name):
        print tny_name, sum(sub_old['tny_name']==tny_name), sum(sub['tny_name']==tny_name)
print a, b

Chennai 0 21
Doha 0 31
Brisbane 0 27
2488 2409


In [16]:
cols = ['s_pct']
df = generate_EM_stats(sub, cols)

s_pct 0.636546532584


In [147]:
stat_history = np.concatenate([sub['p0_s_pct'],sub['p1_s_pct']],axis=0)
tau2_hat = np.nanvar(stat_history)
print tau2_hat

0.0014281577513


In [148]:
stat_history = np.concatenate([sub_old['p0_s_pct'],sub_old['p1_s_pct']],axis=0)
tau2_hat = np.nanvar(stat_history)
print tau2_hat

0.00143649763282


In [6]:
sub[['tny_name','p0_name','p1_name',\
     'p0_s_kls','p0_s_pct_EM','p1_s_kls','p1_s_pct_EM']].loc[79:84]

,tny_name,p0_name,p1_name,p0_s_kls,p0_s_pct_EM,p1_s_kls,p1_s_pct_EM
79,Auckland,Donald Young,Federico Delbonis,0.573700,0.618342,0.589324,0.641060
80,Auckland,Santiago Giraldo,Albert Montanes,0.645451,0.624250,0.660674,0.620087
81,Auckland,Guillermo Garcia,Robin Haase,0.638478,0.612437,0.634435,0.631945
82,Auckland,Marcos Baghdatis,Steve Johnson,0.733536,0.639615,0.692084,0.635690
83,Auckland,Igor Sijsling,Yen Hsun,0.655477,0.648859,0.667468,0.629183
84,Auckland,Bradley Klahn,Daniel Brands,0.746740,0.656862,0.712967,0.653501


In [18]:
sub_old[['tny_name','p0_name','p1_name',\
         'p0_s_kls','p0_s_pct_JS','p1_s_kls','p1_s_pct_JS']].loc[:5]

,tny_name,p0_name,p1_name,p0_s_kls,p0_s_pct_JS,p1_s_kls,p1_s_pct_JS
0,Auckland,Donald Young,Federico Delbonis,0.573700,0.574336,0.589324,0.639250
1,Auckland,Albert Montanes,Santiago Giraldo,0.660674,0.613221,0.645451,0.618632
2,Auckland,Guillermo Garcia,Robin Haase,0.638478,0.604832,0.634435,0.629601
3,Auckland,Steve Johnson,Marcos Baghdatis,0.692084,0.629776,0.733536,0.637896
4,Auckland,Yen Hsun,Igor Sijsling,0.667468,0.624290,0.655477,0.649739
5,Auckland,Bradley Klahn,Daniel Brands,0.746740,0.669786,0.712967,0.655438


In [ ]:
stat_history = np.concatenate([df['p0_s_kls'],df['p1_'+col]],axis=0)

In [111]:
prob_cols = [u'match_prob_kls', u'match_prob_kls_EM', u'match_prob_adj_kls', \
             u'match_prob_adj_kls_EM', u'elo_prob']
for col in prob_cols:
    print col
    print 'accuracy score: ', accuracy_score(1 - sub['winner'], sub[col] >=.5)
    print 'log loss: ',log_loss(1 - sub['winner'], sub[col])

match_prob_kls
accuracy score:  0.648713826367
log loss:  0.647957708155
match_prob_kls_EM
accuracy score:  0.646704180064
log loss:  0.613318223449
match_prob_adj_kls
accuracy score:  0.674839228296
log loss:  0.631435350174
match_prob_adj_kls_EM
accuracy score:  0.674839228296
log loss:  0.621415954217
elo_prob
accuracy score:  0.690514469453
log loss:  0.588863185483


In [110]:
prob_cols = [u'match_prob_kls', u'match_prob_kls_JS', u'match_prob_adj_kls', \
             u'match_prob_adj_kls_JS', u'elo_prob']
for col in prob_cols:
    print col
    print 'accuracy score: ', accuracy_score(sub_old['winner'], sub_old[col] >=.5)
    print 'log loss: ',log_loss(sub_old['winner'], sub_old[col])

match_prob_kls
accuracy score:  0.64798671648
log loss:  0.649029598611
match_prob_kls_JS
accuracy score:  0.654628476546
log loss:  0.615573317508
match_prob_adj_kls
accuracy score:  0.678704856787
log loss:  0.631634020959
match_prob_adj_kls_JS
accuracy score:  0.679119966791
log loss:  0.616994785515
elo_prob
accuracy score:  0.690743046907
log loss:  0.585792277895


In [57]:
sub[['match_year','tny_name','p0_name','p1_name','p0_52_swon','p0_52_svpt',\
     'p1_52_swon','p1_52_svpt']].loc[7]

match_year                 2013
tny_name        Australian Open
p0_name          Kevin Anderson
p1_name       Fernando Verdasco
p0_52_swon                 3191
p0_52_svpt                 4695
p1_52_swon                 2469
p1_52_svpt                 3819
Name: 7, dtype: object

In [58]:
sub_old[['match_year','tny_name','p0_name','p1_name','p0_52_swon','p0_52_svpt',\
     'p1_52_swon','p1_52_svpt']].loc[7]

match_year                 2013
tny_name        Australian Open
p0_name          Kevin Anderson
p1_name       Fernando Verdasco
p0_52_swon                 3191
p0_52_svpt                 4695
p1_52_swon                 2469
p1_52_svpt                 3819
Name: 7, dtype: object

In [ ]:
# YEAR = 2013
# sub = df[(df['p0_name']=='Kevin Anderson') | (df['p1_name']=='Kevin Anderson')]
# sub_old = df_old[(df_old['p0_name']=='Kevin Anderson') | (df_old['p1_name']=='Kevin Anderson')]
# sub = sub[sub['match_year']==YEAR].reset_index()
# sub_old = sub_old[sub_old['match_year']==YEAR].reset_index()